<a href="https://colab.research.google.com/github/IamSantoshKumar/Spark-Workouts/blob/master/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.2/spark-2.3.2-bin-hadoop2.7.tgz
#!tar xf spark-2.3.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
!ls

In [62]:
!pip install pandas

In [0]:
import os
os.chdir("/content/drive/My Drive/Data")
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/drive/My Drive/Data/spark-2.3.2-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[4]").getOrCreate()

In [0]:
!chmod -R 777 '/content/drive/'

In [7]:
df = spark.read.csv('/content/drive/My Drive/testy.txt',inferSchema=True,header=True)
df.show()

+---+----------------+-----------------+-------------+--------------------+--------------------+--------------------+----+-------+---------+--------+------------+
|_c0|Retailer country|Order method type|Retailer type|        Product line|        Product type|             Product|Year|Quarter|  Revenue|Quantity|Gross margin|
+---+----------------+-----------------+-------------+--------------------+--------------------+--------------------+----+-------+---------+--------+------------+
|  0|   United States|              Fax|Outdoors Shop|   Camping Equipment|        Cooking Gear|TrailChef Deluxe ...|2012|Q1 2012| 59628.66|     489|  0.34754797|
|  1|   United States|              Fax|Outdoors Shop|   Camping Equipment|        Cooking Gear|TrailChef Double ...|2012|Q1 2012| 35950.32|     252|   0.4742745|
|  2|   United States|              Fax|Outdoors Shop|   Camping Equipment|               Tents|           Star Dome|2012|Q1 2012| 89940.48|     147|  0.35277197|
|  3|   United States|

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [9]:
df2=df.filter(col('Product type')=='Safety')
df2.show()

+---+----------------+-----------------+-------------+--------------------+------------+--------------------+----+-------+--------+--------+------------+
|_c0|Retailer country|Order method type|Retailer type|        Product line|Product type|             Product|Year|Quarter| Revenue|Quantity|Gross margin|
+---+----------------+-----------------+-------------+--------------------+------------+--------------------+----+-------+--------+--------+------------+
| 15|   United States|              Fax|Outdoors Shop|Mountaineering Eq...|      Safety|Granite Climbing ...|2012|Q1 2012|62464.88|     898|  0.24468085|
| 16|   United States|              Fax|Outdoors Shop|Mountaineering Eq...|      Safety|       Husky Harness|2012|Q1 2012| 34154.9|     559|  0.28363339|
| 17|   United States|              Fax|Outdoors Shop|Mountaineering Eq...|      Safety|Husky Harness Ext...|2012|Q1 2012| 36396.8|     352|  0.47843327|
| 18|   United States|              Fax|Outdoors Shop|Mountaineering Eq...| 

In [10]:
df2.distinct().count()

4

In [15]:
df3=df.groupBy(df.Product).agg({"Revenue": "avg", "Quantity": "max"}).toDF('product','max_quantity','revenue')
df3.show()


+--------------------+------------+---------+
|             product|max_quantity|  revenue|
+--------------------+------------+---------+
|TrailChef Deluxe ...|         632| 68347.37|
|Firefly Rechargea...|        1520|  11673.6|
|             Fairway|         409|  8241.35|
|      Hibernator Pad|         411| 16205.73|
|Husky Harness Ext...|         352|  36396.8|
|Granite Signal Mi...|         126|  4074.84|
|Hailstorm Titaniu...|          75|  65461.5|
|Hailstorm Steel W...|          44| 30095.04|
|  TrailChef Cook Set|        1331| 70400.88|
|             Cat Eye|         101|  4428.85|
|     EverGlow Butane|         109|  6940.03|
|             Maximus|         113|   9325.0|
|         Granite Ice|         333|  25041.6|
|              Trendi|         181|   9104.3|
|         Granite Axe|         856|  32870.4|
|     Firefly Charger|         710|  36494.0|
|             Inferno|         199|  12935.0|
|       TrailChef Cup|        6134| 21434.73|
|Hailstorm Titaniu...|          89

In [0]:
maturity_udf = udf(lambda revenue: "Above Average" if revenue >=50000 else "Below Average", StringType())


In [23]:
df3=df3.withColumn("Class", maturity_udf(df3.revenue))
df3=df3.drop('Rank')
df3.show()

+--------------------+------------+---------+-------------+
|             product|max_quantity|  revenue|        Class|
+--------------------+------------+---------+-------------+
|TrailChef Deluxe ...|         632| 68347.37| Top Customer|
|Firefly Rechargea...|        1520|  11673.6|Below the avg|
|             Fairway|         409|  8241.35|Below the avg|
|      Hibernator Pad|         411| 16205.73|Below the avg|
|Husky Harness Ext...|         352|  36396.8|Below the avg|
|Granite Signal Mi...|         126|  4074.84|Below the avg|
|Hailstorm Titaniu...|          75|  65461.5| Top Customer|
|Hailstorm Steel W...|          44| 30095.04|Below the avg|
|  TrailChef Cook Set|        1331| 70400.88| Top Customer|
|             Cat Eye|         101|  4428.85|Below the avg|
|     EverGlow Butane|         109|  6940.03|Below the avg|
|             Maximus|         113|   9325.0|Below the avg|
|         Granite Ice|         333|  25041.6|Below the avg|
|              Trendi|         181|   91

In [0]:
df3.createGlobalTempView("Revenue")

In [28]:
df4 = spark.sql("select * from global_temp.Revenue as rev where rev.Class='Top Customer'")
df4.show()

+--------------------+------------+----------+------------+
|             product|max_quantity|   revenue|       Class|
+--------------------+------------+----------+------------+
|TrailChef Deluxe ...|         632|  68347.37|Top Customer|
|Hailstorm Titaniu...|          75|   65461.5|Top Customer|
|  TrailChef Cook Set|        1331|  70400.88|Top Customer|
|Hailstorm Titaniu...|          89| 106338.98|Top Customer|
|           Star Dome|         398|  166726.4|Top Customer|
|TrailChef Kitchen...|        3592|  82254.08|Top Customer|
|Granite Climbing ...|         898|  62464.88|Top Customer|
|  Hibernator Extreme|         352|  87728.96|Top Customer|
|     Hibernator Lite|        1415|  85357.44|Top Customer|
|    TrailChef Kettle|       17492| 138711.56|Top Customer|
|Canyon Mule Climb...|         951|  69788.49|Top Customer|
|           Firefly 2|        2899|  76660.11|Top Customer|
|        Star Gazer 6|         172|  134690.2|Top Customer|
|      Husky Rope 100|         227|  739

In [0]:
spark.catalog.dropTempView("Revenue")

In [32]:
df4.describe(['revenue']).show()

+-------+------------------+
|summary|           revenue|
+-------+------------------+
|  count|                24|
|   mean|111182.74500000004|
| stddev| 79789.07616733816|
|    min|          53082.54|
|    max|         417084.48|
+-------+------------------+



In [0]:
df5=df.dropDuplicates(['Product type', 'Product line'])

In [44]:
df5.count()

21

In [45]:
df.dtypes

[('_c0', 'int'),
 ('Retailer country', 'string'),
 ('Order method type', 'string'),
 ('Retailer type', 'string'),
 ('Product line', 'string'),
 ('Product type', 'string'),
 ('Product', 'string'),
 ('Year', 'int'),
 ('Quarter', 'string'),
 ('Revenue', 'double'),
 ('Quantity', 'int'),
 ('Gross margin', 'double')]

In [47]:
df5.select("Product line","Revenue").show()

+--------------------+---------+
|        Product line|  Revenue|
+--------------------+---------+
|   Camping Equipment| 89940.48|
|      Golf Equipment| 25215.25|
|Personal Accessories|   6499.8|
|      Golf Equipment|   4693.5|
|Mountaineering Eq...|  20003.2|
|   Camping Equipment|  8268.41|
|Personal Accessories| 38534.28|
|Personal Accessories|   7520.0|
|  Outdoor Protection|  5718.15|
|   Camping Equipment| 69788.49|
|Personal Accessories|  7015.34|
|Personal Accessories| 30940.25|
|      Golf Equipment| 28434.48|
|Mountaineering Eq...| 62464.88|
|      Golf Equipment|106338.98|
|Mountaineering Eq...|  25041.6|
|   Camping Equipment| 119822.2|
|Mountaineering Eq...| 15122.72|
|  Outdoor Protection|   4057.2|
|   Camping Equipment| 59628.66|
+--------------------+---------+
only showing top 20 rows



In [61]:
df.join(df5, ['Product', 'Product line'],'left').select(df.Product, df['Product line']).limit(1).show()

+--------------------+-----------------+
|             Product|     Product line|
+--------------------+-----------------+
|TrailChef Deluxe ...|Camping Equipment|
+--------------------+-----------------+



In [52]:
df.repartition(10).rdd.getNumPartitions()

10

In [0]:
df6=df.toPandas()

In [64]:
df6.iloc[0:2]

,_c0,Retailer country,Order method type,Retailer type,Product line,Product type,Product,Year,Quarter,Revenue,Quantity,Gross margin
0,0,United States,Fax,Outdoors Shop,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,2012,Q1 2012,59628.66,489,0.347548
1,1,United States,Fax,Outdoors Shop,Camping Equipment,Cooking Gear,TrailChef Double Flame,2012,Q1 2012,35950.32,252,0.474274


In [59]:
df5.withColumnRenamed('Revenue', 'TotalRevenue').show()

+---+----------------+-----------------+----------------+--------------------+--------------------+--------------------+----+-------+------------+--------+------------+
|_c0|Retailer country|Order method type|   Retailer type|        Product line|        Product type|             Product|Year|Quarter|TotalRevenue|Quantity|Gross margin|
+---+----------------+-----------------+----------------+--------------------+--------------------+--------------------+----+-------+------------+--------+------------+
|  2|   United States|              Fax|   Outdoors Shop|   Camping Equipment|               Tents|           Star Dome|2012|Q1 2012|    89940.48|     147|  0.35277197|
| 66|   United States|        Telephone|       Golf Shop|      Golf Equipment|             Putters|   Course Pro Putter|2012|Q1 2012|    25215.25|     349|  0.51695502|
| 32|   United States|              Fax|   Outdoors Shop|Personal Accessories|             Watches|Mountain Man Extreme|2012|Q1 2012|      6499.8|      23|

In [66]:
df5[df5.Product.isin("Star Dome", "Polar Sunn")].show()

+---+----------------+-----------------+-------------+-----------------+------------+---------+----+-------+--------+--------+------------+
|_c0|Retailer country|Order method type|Retailer type|     Product line|Product type|  Product|Year|Quarter| Revenue|Quantity|Gross margin|
+---+----------------+-----------------+-------------+-----------------+------------+---------+----+-------+--------+--------+------------+
|  2|   United States|              Fax|Outdoors Shop|Camping Equipment|       Tents|Star Dome|2012|Q1 2012|89940.48|     147|  0.35277197|
+---+----------------+-----------------+-------------+-----------------+------------+---------+----+-------+--------+--------+------------+



In [67]:
from pyspark.sql import functions as F
df5.select(df5.Product, F.when(df.Quantity > 100, 1).otherwise(0)).show()

+--------------------+--------------------------------------------+
|             Product|CASE WHEN (Quantity > 100) THEN 1 ELSE 0 END|
+--------------------+--------------------------------------------+
|           Star Dome|                                           1|
|   Course Pro Putter|                                           1|
|Mountain Man Extreme|                                           0|
|Course Pro Golf a...|                                           1|
|       Husky Rope 50|                                           1|
|        Firefly Lite|                                           1|
| Glacier GPS Extreme|                                           1|
|       Ranger Vision|                                           0|
|   Sun Shelter Stick|                                           1|
|Canyon Mule Climb...|                                           1|
|           Polar Sun|                                           1|
|        Edge Extreme|                          